In [1]:
import pandas as pd
import numpy as np

np.random.seed(0)

In [2]:
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.feature_selection import SelectPercentile

In [3]:
import sys
import pickle

In [4]:
from util import preprocess, convert_class

[nltk_data] Error loading wordnet: <urlopen error Tunnel connection
[nltk_data]     failed: 302 Moved Temporarily>
[nltk_data] Error loading stopwords: <urlopen error Tunnel connection
[nltk_data]     failed: 302 Moved Temporarily>


In [5]:
# if __name__ == "__main__":
    
#     filename = sys.argv[1]
#     model_path = sys.argv[2]
    
#     train = pd.read_csv(filename)
#     train = preprocess(train, 'Subject')
#     train = preprocess(train, "Content")
#     df = convert_class(train)
    
#     vectorizer = TfidfVectorizer().fit(df["Subject"] + " " + df["Content"])
#     X_train = vectorizer.transform(df["Subject"]+" "+df["Content"])
    
#     vect_path = "vectorizer.pkl"  
#     with open(vect_path, 'wb') as file:  
#         pickle.dump(vectorizer, file)
        
#     clf = BaggingClassifier(base_estimator=SGDClassifier(), random_state=3, n_estimators=12, n_jobs=-3)
#     clf = clf.fit(X_train, df.Class)
    
#     pred = clf.predict(X_train)
#     mat = confusion_matrix(pred, df.Class)
#     total = 0
#     for i in range(mat.shape[0]):
#         total += mat[i][i]/sum(mat[i])

#     print("Micro Accuraccy: ", total/mat.shape[0]) 
#     print("Macro Accuracy: ", np.mean(pred == df.Class))
    
#     with open(model_path, 'wb') as file:  
#         pickle.dump(clf, file)

In [7]:
train = pd.read_csv("train.csv")
train = preprocess(train, 'Subject')
train = preprocess(train, "Content")
df = convert_class(train)

val = pd.read_csv("val.csv")
val = preprocess(val, "Subject")
val = preprocess(val, "Content")
val = convert_class(val)

vectorizer = TfidfVectorizer(ngram_range=(1, 3)).fit(df["Subject"] + " " + df["Content"])
X_train = vectorizer.transform(df["Subject"]+" "+df["Content"])

select = SelectPercentile(percentile=65)
select.fit(X_train, df.Class)
tr = select.transform(X_train)

clf = BaggingClassifier(base_estimator=SGDClassifier(random_state=0), 
                        random_state=0, n_estimators=30, n_jobs=-2)
clf = clf.fit(tr, df.Class)

X_val = select.transform(vectorizer.transform(val["Subject"]+" "+val["Content"]))

pred = clf.predict(X_val)
mat = confusion_matrix(pred, val.Class)
total = 0
for i in range(mat.shape[0]):
    total += mat[i][i]/sum(mat[i])

print("Micro Accuraccy: ", total/mat.shape[0], "Macro Accuracy: ", np.mean(pred == val.Class))
print("Score", (total/mat.shape[0] + np.mean(pred == val.Class))/2)

Micro Accuraccy:  0.8403099285709035 Macro Accuracy:  0.827433628318584
Score 0.8338717784447438


In [ ]:
# train = pd.read_csv("train.csv")
# train = preprocess(train, 'Subject')
# train = preprocess(train, "Content")
# df = convert_class(train)

# val = pd.read_csv("val.csv")
# val = preprocess(val, "Subject")
# val = preprocess(val, "Content")
# val = convert_class(val)

# vectorizer = TfidfVectorizer(ngram_range=(1, 3)).fit(df["Subject"] + " " + df["Content"])
# X_train = vectorizer.transform(df["Subject"]+" "+df["Content"])

# select = SelectPercentile(percentile=65)
# select.fit(X_train, df.Class)
# tr = select.transform(X_train)

# clf = BaggingClassifier(base_estimator=SGDClassifier(random_state=0), 
#                         random_state=0, n_estimators=30, n_jobs=-2)
# clf = clf.fit(tr, df.Class)

# X_val = select.transform(vectorizer.transform(val["Subject"]+" "+val["Content"]))

# pred = clf.predict(X_val)
# mat = confusion_matrix(pred, val.Class)
# total = 0
# for i in range(mat.shape[0]):
#     total += mat[i][i]/sum(mat[i])

# print("Micro Accuraccy: ", total/mat.shape[0], "Macro Accuracy: ", np.mean(pred == val.Class))
# print("Score", (total/mat.shape[0] + np.mean(pred == val.Class))/2)